In [1]:
import math

import numpy
import pandas

import ipywidgets as widgets
from IPython.display import display, clear_output

import gensim
from gensim.corpora import Dictionary
from gensim.models import TfidfModel
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

import pythainlp

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans

from data_tokenizer import load_corpus

from model.upgrade_sdc import UpgradeSDC
from model.sdc import SDC

### Load Data

In [2]:
file_name = 'ผู้บริโภค - TrueCoffee.txt'

corpus, labels = load_corpus('../data/facebook/' + file_name)

len_corpus = len(corpus)
print('Total documents', len_corpus)

clusters = list(set(labels))
print(len(clusters), 'clusters')

f = open('../data/facebook/tokenized/tokenized_' + file_name)
tokenized_corpus = eval(f.read())
f.close()

Total documents 350
1 clusters


### Preprocess Corpus

#### Remove Words

In [3]:
dictionary = Dictionary(tokenized_corpus)
print('origin:', len(dictionary), 'words')

dictionary.filter_extremes(no_below=2, no_above=0.7, keep_n=len(dictionary))
print('filter frequent words:', len(dictionary), 'words')

letter_words = [id for id in range(len(dictionary)) if len(dictionary[id]) <= 1] 
dictionary.filter_tokens(bad_ids=letter_words)
print('filter letter words:', len(dictionary), 'words')

stopwords = pythainlp.corpus.stopwords.words('thai')
stopwords.append('นี้')
dictionary.add_documents([stopwords])
stopwords = [dictionary.token2id[word] for word in stopwords]
dictionary.filter_tokens(bad_ids=stopwords)
print('filter stop words:', len(dictionary), 'words')

origin: 948 words
filter frequent words: 370 words
filter letter words: 368 words
filter stop words: 219 words


In [4]:
idx_corpus = [dictionary.doc2idx(doc) for doc in tokenized_corpus]

temp_corpus = []
for doc in idx_corpus:
    temp_corpus.append([dictionary[id] for id in doc if id >= 0])
idx_corpus = temp_corpus

#### Dimension Reduction

In [5]:
average_doc_size = 0
for doc in idx_corpus:
    average_doc_size += len(doc)
average_doc_size /= len(idx_corpus)
average_doc_size = math.ceil(average_doc_size)

df = dictionary.dfs
filtered_corpus = []
for doc in idx_corpus:
    new_doc = [(word, df[dictionary.token2id[word]]) for word in doc]
    new_doc.sort(reverse=True, key=lambda x: x[1])
    new_doc = new_doc[:average_doc_size]
    filtered_corpus.append([word for word, df in new_doc])

#### Doc2Vec

In [6]:
tagged_corpus = [TaggedDocument(doc, [i]) for i, doc in enumerate(idx_corpus)]
model = Doc2Vec(tagged_corpus, vector_size=average_doc_size, window=4, min_count=2, epochs=100)
model.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)

paragraph_vectors = [model.infer_vector(doc) for doc in idx_corpus]
paragraph_vectors = pandas.DataFrame(paragraph_vectors, dtype=float)

### Clustering

In [7]:
def get_onehot(corpus):
    new_dict = Dictionary(corpus)
#     new_dict.filter_extremes(no_below=2, no_above=1, keep_n=len(new_dict))

    unique_words = [new_dict[id] for id in range(len(new_dict))]
    array = numpy.zeros((len_corpus, len(unique_words)), dtype=float)    
    for i, doc in enumerate(corpus):
        for word in doc:
            array[i, new_dict.token2id[word]] += 1

        ## normalization
        if len(doc) != 0:
#             array[i] = numpy.divide(array[i], len(idx_corpus[i]))
            array[i] = numpy.divide(array[i], len(doc))
    
    return pandas.DataFrame(array, columns=unique_words, dtype=float)

In [8]:
def generate_result(predicted_labels):
    result = pandas.DataFrame()
    result['comment'] = corpus
    result['tokenized_comment'] = filtered_corpus
    result['label'] = labels
    result['predicted_label'] = predicted_labels
    return result

In [9]:
def eval_cluster(onehot_corpus, result):
    label_count = numpy.unique(result['predicted_label'])
    num_cluster = label_count[-1] + 1

    clusters = [[] for i in range(num_cluster)]
    corpus_centroid = []
    for i, label in result['predicted_label'].iteritems():
        clusters[label].append(numpy.array(onehot_corpus.iloc[i]))
        corpus_centroid.append(numpy.array(onehot_corpus.iloc[i]))
    corpus_centroid = numpy.mean(corpus_centroid, axis=0).reshape(1, -1)   

#     print('\tIntra cluster sim\tInter cluster sim\tIntra / Inter')
    compactness = 0
    centroids = []
    for i in range(num_cluster):
        size = len(clusters[i])
        if size != 0:
            centroid = numpy.mean(clusters[i], axis=0)
            centroids.append(centroid)
            centroid = centroid.reshape(1, -1)
            similarities = cosine_similarity(centroid, clusters[i])
            compactness += numpy.sum(similarities)

#             intra = numpy.sum(similarities) / size
#             inter = cosine_similarity(centroid, corpus_centroid)[0][0]
#             print(i, end='\t')
#             print(intra, end='\t')
#             print(inter, end='\t')
#             print(intra / inter)
    return compactness, centroids

In [11]:
min_samples = 7
eps = 0.32
epoch = 1

# onehot_corpus = get_onehot(idx_corpus)
onehot_corpus = get_onehot(filtered_corpus)

max_compactness = 0
for i in range(epoch):
#     model = UpgradeSDC()
#     _tpredicted_labels, _tmarks = model.predict(onehot_corpus, min_samples, eps)
    
    _tmarks = None
#     model = DBSCAN(metric='cosine', eps=eps, min_samples=min_samples).fit(onehot_corpus)
#     _tpredicted_labels = model.labels_ + 1

    model = KMeans(n_clusters=7).fit(onehot_corpus)
    _tpredicted_labels = model.labels_
    
    _tresult = generate_result(_tpredicted_labels)
    compactness, _tcentroids = eval_cluster(onehot_corpus, _tresult)
    
    if compactness > max_compactness:
        max_compactness = compactness
        predicted_labels = _tpredicted_labels
        marks = _tmarks
        result = _tresult
        centroids = _tcentroids
        
print(max_compactness)
label_count = numpy.unique(result['predicted_label'], return_counts=True) 
num_cluster = label_count[0][-1] + 1
print(label_count, '\n')

138.77649855185808
(array([0, 1, 2, 3, 4, 5, 6], dtype=int32), array([ 56,  28, 216,  13,  10,  25,   2])) 



In [12]:
sims = cosine_similarity(centroids)
new_labels = [i for i in range(num_cluster)]
print(new_labels)
for i, row in reversed(list(enumerate(sims))):
    for j, value in reversed(list(enumerate(row[:i + 1]))):
        if i != j and value >= eps - eps / 20:
            print(i, j, value)
            new_labels = [new_labels[j] if label == new_labels[i] else label for label in new_labels]
print(new_labels)

grouped_labels = numpy.zeros(len_corpus)
for i, label in enumerate(predicted_labels):
    grouped_labels[i] = new_labels[label]
new_result = generate_result(grouped_labels)

[0, 1, 2, 3, 4, 5, 6]
[0, 1, 2, 3, 4, 5, 6]


### Result

In [13]:
class Widget:
    def __init__(self, result, marks, column_name):
        self.result = result
        self.column_name = column_name
        self.marks = marks
        
        label_count = numpy.unique(result['predicted_label'])
        self.widget = widgets.ToggleButtons(
            options=[int(num) for num in label_count],
            disabled=False,
            button_style='',
        )
        
        self.widget.observe(self.on_click, names='index')
        self.on_click({'new' : 0})
        
    def on_click(self, change):
        clear_output()
        display(self.widget)
        new = self.widget.options[change['new']]
        for index, value in self.result[self.result['predicted_label'] == new][self.column_name].iteritems():
            if self.marks:
                if index in self.marks[0]:
                    print("@", end="")
                elif index in self.marks[1]:
                    print("*", end="")
            print(index, value)

In [14]:
w1 = Widget(new_result, marks, 'comment')

ToggleButtons(index=2, options=(0, 1, 2, 3, 4, 5, 6), value=2)

1 เรางง สั่งคาปูชิโนหวานน้อย ทำไม่ได้ค่ะ ถ้าลูกค้าอยากได้หวานน้อยต้องสั่งลาเต้ กูงงงงงงง
2 ไม่ควรส่งโปรมาให้เรา​ ถ้าสิทธิ์​มันเต็มแล้วจากใจลูกค้าทรูมูฟ
4 ซื้อลิขสิทธิ์ UCL มาด้วยนะ จ่ายรายเดือนไป แต่ไม่มีให้ดู
5 ช่วงนี้เนตช้ามากเลยครับ ช่วยปรับปรุงด้วยครับ รายเดือนแพงแต่ใช้ไม่คุ้ม
7 เน็ตบ้านหลุดบ่อยมากครับลมพัดทีหาย นี่เน็ตรึโพยหวย #ปรับปรุงด้วยคนับ
8 ก่อนลูกค้าจะสั่ง รบกวนพนักงานอ้าปากบอกเลยค่ะ “สิทธิ์เต็ม” เบื่อมากกก
9 มี point มีโปรก็ไม่ได้ดืม สิทธิ์เต็ม ทำไงล่ะ ขอบคุณครับ ไปกินสตาบัคดีกว่า ไมอยากเป็นเหยื่อยการค้า
10 เมนูชาไทยจะน้ำแข็งแน่นแก้วอะไรขนาดนั้นกินแป๊บเดียวก็หมดแก้ว เหลือแต่น้ำแข็งเปล่าเต็มแก้ว
11 อร่อยดีเเล้ว​ต้อง​ #Signature นะครับอย่าลืมใช้Black cardนะครับผม
13 ใช้​ code  ทีไรเต็มตลอด​ พนักงานแนะนำให้กด​ code  ไว้ตอนเที่ยงคืน​ )​
15 ชาเย็นอร่อย ชอบที่เป็นน้ำแข็งปั่นละเอียด แต่ขอน้ำเยอะกว่านี้ได้ไหม ดูไปสองปื้ดหมดแหล่ว ราคาโหดร้ายไป๊
16 โปรติดสัญญานี่เนตใช้ดีมาก ดูยูทูปเท่าไหร่ก็ไม่หมด พอครบสัญญาปุ๊บ อีดอกแค่สไล้เฟชบุค ไม่ถึงเดือนเนตปลิวหายไปอย่างกะโดนปล้น
17 Wi-Fi ทรูที

In [15]:
w2 = Widget(result, marks, 'comment')

ToggleButtons(options=(0, 1, 2, 3, 4, 5, 6), value=0)

0 ร้านสวยมาก สวยจนอยากบอกว่า เอาเงินแต่งร้าน ส่ง พนง ไปเรียนชงกาแฟก่อนเถอะค่ะ
6 ไม่ ชอบ การ แบ่ง ชนชั้น ในร้านกาแฟ ผู้ถือบัตรอีกแบบ นั่งได้แค่ตรงนี้ ผู้ถือบัตรอีกแบบนั่งตรงนี้ จ่ายราคากาแฟก็เท่ากันไหม
12 ผมว่า ผมชอบรสชาติของกาแฟคุณ หลายๆแก้วประทับใจกว่า แบรนด์นางเงือกแต่ว่า แพงไปหว่ะ ไม่เหมาะกะคนไทย (ถึงแม้จะใช้ส่วนลด ลูกค้าทรู แล้วก้ตาม)สังเกตได้ว่า น้อยสาขา ที่คนจะแน่น หรือต้องต่อคิวสัีงกาแฟ เหมือนแบรนด์นางเงือก…See more
14 ถ้า》 กาแฟจืด เหมือนน้ำเปล่า แสดงว่าชอบกิน สายพันธุ์โรบัสต้า (เบอดี้,3 in 1)ถ้า 》 แพง ให้เปิด Promotion มี 10 กว่าโปร จนไม่รู้จะใช้โปรไหน จน งง …See more
19 ย้ายค่ายจากทรูมาทุกเบอร์ เพราะสัญญาณเหรี้ยมาก เดี๋ยววว~~ เกี่ยวไรกะกาแฟ 🤣
20 นี้จะเหมือนอเมซอน แต่ละสาขารสชาติไม่เหมือนกัน คนทำกาแฟเปลี่ยนรสชาติก็เปลี่ยน ปล.เมิงควรให้ลูกค้าที่ใช้ทรูโดยเฉพาะได้สิทธิ์ได้ส่วนลดพิเศษบ้าง
30 ทรูพอยน์ยังใช้ได้อยู่มั้ย มีอยู่ 3,000 กว่า อยากกินกาแฟระหว่างรอจ่ายค่าโทรศัพท์มั่งอ่ะ
45 บอกพนักงงานชงกาแฟทำหน้าตาดีๆ เลิกกระแทกแก้วสักที เวลาส่งคืนลูกค้าเยอะๆอ่ะ ละก็ช่วยทำคอมเม้นลูกค้าให้มัน

In [16]:
w3 = Widget(result, marks, 'tokenized_comment')

ToggleButtons(options=(0, 1, 2, 3, 4, 5, 6), value=0)

0 ['กาแฟ', 'ร้าน', 'ชง', 'สวย', 'สวย']
6 ['กาแฟ', 'กาแฟ', 'ราคา', 'ร้าน', 'ชอบ']
12 ['กาแฟ', 'แพง', 'รสชาติ', 'สาขา', 'เหมือน']
14 ['กาแฟ', 'แพง', 'กิน', 'โปร', 'โปร']
19 ['กาแฟ', 'สัญญาณ', 'ย้าย', 'ค่าย', 'เกี่ยว']
20 ['กาแฟ', 'รสชาติ', 'รสชาติ', 'สาขา', 'เหมือน']
30 ['กาแฟ', 'กิน', 'จ่าย', 'รอ', 'ค่า']
45 ['กาแฟ', 'ดี', 'แก้ว', 'ลูกค้า', 'ลูกค้า']
47 ['กาแฟ', 'เต็ม', 'ทรู', 'สิทธิ์', 'สิทธิ์']
50 ['กาแฟ', 'ทรู', 'ทำ', 'ทำ', 'คุณภาพ']
55 ['กาแฟ', 'ปรับปรุง', 'เหมือน', 'เน็ต', 'เรื่อง']
57 ['กาแฟ', 'ร้าน', 'น้ำ', 'แดก', 'แม่ง']
58 ['กาแฟ', 'กาแฟ', 'อร่อย', 'ราคา', 'ราคา']
59 ['กาแฟ', 'เหมือน', 'โปร', 'เรื่อง', 'เรื่อง']
62 ['กาแฟ', 'กิน', 'ร้าน', 'ทรู', 'มือถือ']
74 ['กาแฟ', 'ราคา', 'แพง', 'กิน', 'เหมือน']
90 ['กาแฟ', 'ราคา', 'ร้าน']
91 ['กาแฟ', 'น้ำ', 'สาขา', 'เต็ม', 'แก้ว']
99 ['กาแฟ', 'ดี', 'ขาย', 'ลอง']
101 ['กาแฟ', 'แรง', 'แม่ง']
114 ['กาแฟ', 'จืด', 'เปรี้ยว']
122 ['กาแฟ', 'เรื่อง', 'ดู', 'รู้', 'รู้']
128 ['กาแฟ', 'ดี', 'ดู']
136 ['กาแฟ', 'จ่าย', 'ค่า', 'กลิ่น', 'หอม']
143 ['กาแฟ

In [17]:
seed = 164
compare = 145

a = numpy.array(onehot_corpus.iloc[seed]).reshape(1, -1)
b = numpy.array(onehot_corpus.iloc[compare]).reshape(1, -1)
print(cosine_similarity(a,b))
print(filtered_corpus[seed])
print(filtered_corpus[compare])

# print(sims[12])

[[0.31622777]]
['รสชาติ', 'งั้น']
['ราคา', 'แพง', 'รสชาติ', 'ร้าน', 'สวย']
